In [1]:
import torch

## Tensors
are multi-dimensional arrays that can use GPUs, distribute operations on multiple machines, keep track of graph computations that created them, etc.

In [5]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [10]:
print(a[0])
print(a[1:])
print(float(a[0]))
a[1] = 2
print(a)

tensor(1.)
tensor([1., 1.])
1.0
tensor([1., 2., 1.])


Tensors are views over typically contiguous memory blocks containing unboxed C numeric types (unlike Python lists which are collections of objs individually allocated in memory)

In [14]:
points = torch.zeros(6)
points[0] = 4
points[1] = 5
points[-1] = 1
points

tensor([4., 5., 0., 0., 0., 1.])

In [19]:
points = torch.tensor([4.0, 5, 0, 0, 0, 1])
print(points[0], points[1])
float(points[0]), float(points[1])

tensor(4.) tensor(5.)


(4.0, 5.0)

In [21]:
l = [[4.0, 5], [0, 0], [0, 1]]
points = torch.tensor(l)
points

tensor([[4., 5.],
        [0., 0.],
        [0., 1.]])

In [23]:
points.shape

torch.Size([3, 2])

Slicing presents different *views*; doesn't allocate anything new.

In [41]:
print(points[1:])
print(points[0, 1:])
print(points[0, 1])
print(points[0, :])
print(points[:, 0])
print(points[:2, 1])
print(points[None])
print(points[None].shape) # adds a dim

tensor([[0., 0.],
        [0., 1.]])
tensor([5.])
tensor(5.)
tensor([4., 5.])
tensor([4., 0., 0.])
tensor([5., 0.])
tensor([[[4., 5.],
         [0., 0.],
         [0., 1.]]])
torch.Size([1, 3, 2])


In [24]:
new_pts = torch.zeros(points.shape)
new_pts

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

### Named tensors

In [44]:
img_t = torch.rand(3, 5, 5) # [channels, rows, cols]
weights = torch.tensor([0.2126, 0.7152, 0.0722])
batch_t = torch.rand(2, 3, 5, 5)

In [46]:
img_gray_naive = img_t.mean(-3)
img_gray_naive

tensor([[0.5866, 0.4663, 0.6649, 0.9143, 0.5160],
        [0.5169, 0.4241, 0.5714, 0.2131, 0.5512],
        [0.6316, 0.5053, 0.5126, 0.5254, 0.4778],
        [0.5053, 0.7308, 0.5167, 0.5006, 0.8138],
        [0.6233, 0.6801, 0.4599, 0.3800, 0.4546]])

In [47]:
batch_gray_naive = batch_t.mean(-3)
batch_gray_naive

tensor([[[0.4095, 0.5322, 0.7011, 0.4027, 0.2943],
         [0.7517, 0.5483, 0.4285, 0.4201, 0.5372],
         [0.3643, 0.5345, 0.3738, 0.3978, 0.5396],
         [0.5343, 0.3456, 0.4158, 0.5098, 0.2328],
         [0.3597, 0.2754, 0.3827, 0.7999, 0.5019]],

        [[0.3086, 0.3740, 0.2831, 0.4915, 0.5055],
         [0.7231, 0.7809, 0.4159, 0.4591, 0.6856],
         [0.4263, 0.3111, 0.7109, 0.6151, 0.6266],
         [0.5521, 0.4347, 0.4463, 0.4084, 0.3361],
         [0.2912, 0.3716, 0.6864, 0.3930, 0.4974]]])

In [49]:
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [52]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1) # returns a new tensor with a dimension of size 1 inserted at the end
weights.shape, unsqueezed_weights.shape

(torch.Size([3]), torch.Size([3, 1, 1]))

In [54]:
img_t.shape, batch_t.shape

(torch.Size([3, 5, 5]), torch.Size([2, 3, 5, 5]))

In [67]:
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
print(img_weights.shape, batch_weights.shape)
img_gray_weighted = img_weights.sum(-3) # sum along rgb dim of tensor (so sum all rgb values)
batch_gray_weighted = batch_weights.sum(-3)
img_gray_weighted.shape, batch_gray_weighted.shape, unsqueezed_weights.shape

torch.Size([3, 5, 5]) torch.Size([2, 3, 5, 5])


(torch.Size([5, 5]), torch.Size([2, 5, 5]), torch.Size([3, 1, 1]))

Using named tensors (experimental)

In [68]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

C:\Users\ncris\anaconda3\envs\deep_learn\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1408.)
  """Entry point for launching an IPython kernel.


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [73]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
print(img_named.shape, img_named.names)
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print(batch_named.shape, batch_named.names)

torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [78]:
weights_aligned = weights_named.align_as(img_named) # missing dimensions added and existing ones permuted to the right order
print(weights.shape, weights.names)
print(weights_aligned.shape, weights_aligned.names)

torch.Size([3]) (None,)
torch.Size([3, 1, 1]) ('channels', 'rows', 'columns')


In [82]:
gray_named = (img_named * weights_aligned).sum('channels') # (img_named * weights_aligned).shape is same as img_named.shape
img_named.shape, weights_aligned.shape, gray_named.shape, gray_named.names

(torch.Size([3, 5, 5]),
 torch.Size([3, 1, 1]),
 torch.Size([5, 5]),
 ('rows', 'columns'))

In [93]:
(img_named[..., :3] * weights_named) # error bc weights_named not aligned

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.

In [94]:
gray_plain = gray_named.rename(None)
gray_plain.names

(None, None)

Unfortunately I don't think named tensors are in wide usage--stick to regular operations for now.

### Tensor element types
All elements in tensor must be of same type, specified with dtype. The default is 32-bit floating-point.

In [96]:
pts = torch.ones(3, 4, 5)
pts.dtype

torch.float32

In [97]:
pts = torch.tensor([2,2]) # will be int64 if created with int arguments
pts.dtype

torch.int64

In [99]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [100]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [107]:
pts_64 = torch.rand(5, dtype=torch.double)
pts_short = pts_64.to(dtype=torch.short)
print(pts_64, pts_short)
print(pts_64 * pts_short)

tensor([0.5771, 0.2242, 0.4326, 0.4750, 0.7197], dtype=torch.float64) tensor([0, 0, 0, 0, 0], dtype=torch.int16)
tensor([0., 0., 0., 0., 0.], dtype=torch.float64)


Other operations

In [110]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1) # swap dims 0 and 1
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [112]:
a.shape, a.transpose(0, 1).shape

(torch.Size([3, 2]), torch.Size([2, 3]))

### Storage
Values in tensors are allocated in contiguous chunks of memory managed by torch.Storage instances, which are 1D arrays of numerical data. A tensor instance is a view of a Storage instance that can index into it.

In [118]:
pts_storage = pts.storage()
pts_storage

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [119]:
pts_storage[0]

4.0

In [ ]:
pts_storage[0] = 10
pts

in-place operations have a trailing underscore like ```zero_```

In [124]:
pts.zero_()
pts

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

### Size, offset, stride
- size (shape) is number of elements across each dim
- storage offset is index in storage corresponding to first element in the tensor
- stride is number of elements to be skipped over to obtain the next element along each dim

In [131]:
pts = torch.tensor([[4, 1], [5, 3], [2, 1]], dtype=torch.float32)
second_pt = pts[1]
print(pts)
print(second_pt)
print(second_pt.storage_offset())

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([5., 3.])
2


In [133]:
second_pt.shape

torch.Size([2])

```second_pt``` has offset 2 in storage as we need to skip both elements in the first row of ```pts```.

In [134]:
pts.stride()

(2, 1)

Stride is the number of elements that need to be skipped when the index is increased by 1 in the corresp dimension.

In this example to get to a new row, need to skip 2 elements; to get to a new column, need to skip 1 element.

Indexing scheme for element ```i, j``` in a 2D tensor: ```storage_offset + stride[0] * i + stride[1] * j```

In [140]:
second_pt.shape, second_pt.storage_offset(), second_pt.stride()

(torch.Size([2]), 2, (1,))

In [141]:
second_pt[0] = 10
second_pt, pts

(tensor([10.,  3.]),
 tensor([[ 4.,  1.],
         [10.,  3.],
         [ 2.,  1.]]))

In [143]:
second_pt = pts[1].clone()
second_pt[0] = 100
second_pt, pts

(tensor([100.,   3.]),
 tensor([[ 4.,  1.],
         [10.,  3.],
         [ 2.,  1.]]))

Transposing

In [146]:
pts_t = pts.t()
pts, pts_t

(tensor([[ 4.,  1.],
         [10.,  3.],
         [ 2.,  1.]]),
 tensor([[ 4., 10.,  2.],
         [ 1.,  3.,  1.]]))

In [148]:
id(pts.storage()) == id(pts_t.storage())

True

In [149]:
print(pts.shape, pts_t.shape)
print(pts.stride(), pts_t.stride())

torch.Size([3, 2]) torch.Size([2, 3])
(2, 1) (1, 2)


In [155]:
some_t = torch.ones(3, 4, 5)
some_t_transpose = some_t.transpose(0, 2)
print(some_t.shape, some_t_transpose.shape)
print(some_t.stride(), some_t_transpose.stride())

torch.Size([3, 4, 5]) torch.Size([5, 4, 3])
(20, 5, 1) (1, 5, 20)


Contiguous tensors

In [157]:
pts.is_contiguous(), pts_t.is_contiguous()

(True, False)

In [160]:
pts_t_cont = pts_t.contiguous()
pts_t_cont, pts_t_cont.stride()

(tensor([[ 4., 10.,  2.],
         [ 1.,  3.,  1.]]),
 (3, 1))

In [162]:
pts.storage()

 4.0
 1.0
 10.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [161]:
pts_t_cont.storage()

 4.0
 10.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]

## Using GPUs
every tensor can be transferred to one of the GPUs to perform parallel, fast computations. The below doesn't work because I don't have a CUDA GPU. Tensor is stored in RAM of GPU.

In [163]:
pts_gpu = torch.tensor([[4, 1], [5, 3], [2, 1]], device='cuda')
pts_gpu = pts.to(device='cuda')
pts_gpu = pts.to(device='cuda:0') # if using multiple GPUs
pts_cpu = pts_gpu.to(device='cpu')

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

## Numpy integration

In [164]:
pts_np = pts.numpy()
pts_np

array([[ 4.,  1.],
       [10.,  3.],
       [ 2.,  1.]], dtype=float32)

In [167]:
pts = torch.from_numpy(pts_np)
pts

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

## Serializing Tensors
pytorch uses pickle under the hood (but not interoperable--can only be re-opened with pytorch in this way)

In [168]:
torch.save(pts, 'data/p1ch3/ourpoints.t')

In [169]:
pts = torch.load('data/p1ch3/ourpoints.t')

In [171]:
pts

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

See the end of the chapter for saving tensors interoperably using HDF5 format.

## Exercises
a)

In [173]:
a_list = list(range(9))
a = torch.tensor(a_list)
a.shape, a.stride(), a.storage_offset() # shape should be [9], stride should be (1), storage_offset should be 0

(torch.Size([9]), (1,), 0)

In [175]:
b = a.view(3, 3)
b

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [178]:
b.shape, b.stride(), b.storage_offset() # shape should be (3, 3), stride should be (3, 1), storage_offset should be 0

(torch.Size([3, 3]), (3, 1), 0)

In [176]:
id(a.storage()) == id(b.storage())

True

In [177]:
c = b[1:, 1:]
c.shape, c.stride(), c.storage_offset() # shape should be (2, 2), stride should be (2, 1), storage_offset should be 4

(torch.Size([2, 2]), (3, 1), 4)

Stride actually (3, 1) because still have to pass over elements not in c but still in b (like 6).

b)

In [191]:
a.cos()

tensor([ 1.0000,  0.5403, -0.4161, -0.9900, -0.6536,  0.2837,  0.9602,  0.7539,
        -0.1455])

In [182]:
print(b)
print(torch.cos(b))

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
tensor([[ 1.0000,  0.5403, -0.4161],
        [-0.9900, -0.6536,  0.2837],
        [ 0.9602,  0.7539, -0.1455]])


In [199]:
a.sqrt()
a.float().sqrt_()

tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])